In [1]:
# script to model avergage trip distances in all cities
# last update Peter Berrill June 21 2023


# load in required packages
import numpy as np
import pandas as pd
import shap
import re
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, cross_validate, GroupKFold, StratifiedGroupKFold, RepeatedKFold, StratifiedKFold, GridSearchCV, KFold
from sklearn import metrics, linear_model
from xgboost import XGBClassifier, XGBRegressor
import os
import sys
import matplotlib.pyplot as plt
import pickle
import statsmodels.formula.api as smf
from datetime import datetime

cities_all=['Berlin','Dresden','Düsseldorf','Frankfurt am Main','Kassel','Leipzig','Magdeburg','Potsdam','Clermont','Dijon','Lille','Lyon','Montpellier','Nantes','Nimes','Paris','Toulouse','Madrid','Wien','France_other','Germany_other']
countries=['Germany','Germany','Germany','Germany','Germany','Germany','Germany','Germany','France','France','France','France','France','France','France','France','France','Spain','Austria','France','Germany']



pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


In [2]:
city='Berlin'
country=countries[cities_all.index(city)]
df=pd.read_csv('../outputs/Combined/' + city + '_UF.csv',dtype={'Ori_geocode': str, 'Des_geocode': str,'Res_geocode': str })


In [3]:
df['Commute_Trip']=0
df.loc[df['Trip_Purpose_Agg']=='Home↔Work','Commute_Trip']=1
# df_UF=df.loc[:,['Res_geocode', 'HH_PNR','Age','Sex','Occupation','Education',
#                 'DistSubcenter_res', 'DistCenter_res',
#                 #'PopDensity_res','BuildDensity_res',
#                 'UrbPopDensity_res', 'UrbBuildDensity_res',
#                 'IntersecDensity_res', 'street_length_res', 'LU_UrbFab_res','bike_lane_share_res',
#                 'LU_Comm_res', 'Commute_Trip','Trip_Distance']]

In [7]:
commute_share=pd.DataFrame(df.groupby(['HH_PNR'])['Commute_Trip'].mean()).reset_index()
commute_share.head(25)

,HH_PNR,Commute_Trip
0,102956_1,1.000000
1,102960_1,0.000000
2,102960_2,0.000000
3,102964_1,1.000000
4,102964_2,1.000000
5,102964_3,0.000000
6,102964_4,0.000000
7,102966_1,0.333333
8,102966_2,1.000000
9,102976_1,0.000000


In [10]:
commute_exists=commute_share.copy()
commute_exists['Commute_Trip']=np.ceil(commute_exists['Commute_Trip'])
commute_exists

,HH_PNR,Commute_Trip
0,102956_1,1.0
1,102960_1,0.0
2,102960_2,0.0
3,102964_1,1.0
4,102964_2,1.0
...,...,...
31975,792139_2,0.0
31976,792145_1,1.0
31977,792145_2,1.0
31978,792145_3,1.0


In [43]:
commute_share=pd.DataFrame(df.groupby(['HH_PNR'])['Commute_Trip'].mean()).reset_index()
commute_exists=commute_share.copy()
commute_exists['Commute_Trip']=np.ceil(commute_exists['Commute_Trip'])
categ=df[['HH_PNR','Season','Age','Sex','Occupation','Education','CarOwnershipHH']].drop_duplicates()
df_trip_daily=pd.DataFrame(df.groupby(['Res_geocode','HH_PNR'])['Trip_Distance'].sum())
df_trip_daily.reset_index(inplace=True,drop=False)
df_UF_Features=df.loc[:,['Res_geocode', 
                'DistSubcenter_res', 'DistCenter_res',
                'UrbPopDensity_res', 'UrbBuildDensity_res',
                'IntersecDensity_res', 'street_length_res', 'LU_UrbFab_res','bike_lane_share_res',
                'LU_Comm_res',]].groupby('Res_geocode').mean().drop_duplicates() 
df_UF_Features.reset_index(inplace=True)

In [46]:
df_UF_Features=df.loc[:,['Res_geocode', 
                'DistSubcenter_res', 'DistCenter_res',
                'UrbPopDensity_res', 'UrbBuildDensity_res',
                'IntersecDensity_res', 'street_length_res', 'LU_UrbFab_res','bike_lane_share_res',
                'LU_Comm_res',]].groupby('Res_geocode').mean().drop_duplicates() 
df_UF_Features.reset_index(inplace=True)

In [49]:
df_UF=df_UF_Features.merge(df_trip_daily)
df_UF

,Res_geocode,DistSubcenter_res,DistCenter_res,UrbPopDensity_res,UrbBuildDensity_res,IntersecDensity_res,street_length_res,LU_UrbFab_res,bike_lane_share_res,LU_Comm_res,HH_PNR,Trip_Distance
0,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102956_1,1458.0
1,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102964_1,12518.0
2,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102964_2,2948.0
3,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102964_3,9368.0
4,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102964_4,9368.0
...,...,...,...,...,...,...,...,...,...,...,...,...
31975,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,788821_1,41674.0
31976,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,791203_1,2846.0
31977,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,791203_2,3985.0
31978,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,792023_1,3975.0


In [50]:
df_UF=df_UF.merge(categ)
df_UF

,Res_geocode,DistSubcenter_res,DistCenter_res,UrbPopDensity_res,UrbBuildDensity_res,IntersecDensity_res,street_length_res,LU_UrbFab_res,bike_lane_share_res,LU_Comm_res,HH_PNR,Trip_Distance,Season,Age,Sex,Occupation,Education,CarOwnershipHH
0,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102956_1,1458.0,Spring,29,2,Employed_PartTime,University,0
1,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102964_1,12518.0,Winter,46,1,Employed_PartTime,University,0
2,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102964_2,2948.0,Winter,46,2,Employed_PartTime,University,0
3,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102964_3,9368.0,Winter,12,1,Student_School,Elementary,0
4,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102964_4,9368.0,Winter,9,1,Student_School,No diploma yet,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31975,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,788821_1,41674.0,Winter,68,2,Retired,University,1
31976,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,791203_1,2846.0,Winter,83,1,Retired,University,1
31977,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,791203_2,3985.0,Winter,78,2,Retired,Secondary,1
31978,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,792023_1,3975.0,Winter,78,2,Retired,Secondary,1


In [51]:
#df_UF=df_UF.merge(commute_share)
df_UF=df_UF.merge(commute_exists)

In [52]:
df_UF

,Res_geocode,DistSubcenter_res,DistCenter_res,UrbPopDensity_res,UrbBuildDensity_res,IntersecDensity_res,street_length_res,LU_UrbFab_res,bike_lane_share_res,LU_Comm_res,HH_PNR,Trip_Distance,Season,Age,Sex,Occupation,Education,CarOwnershipHH,Commute_Trip
0,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102956_1,1458.0,Spring,29,2,Employed_PartTime,University,0,1.0
1,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102964_1,12518.0,Winter,46,1,Employed_PartTime,University,0,1.0
2,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102964_2,2948.0,Winter,46,2,Employed_PartTime,University,0,1.0
3,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102964_3,9368.0,Winter,12,1,Student_School,Elementary,0,0.0
4,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102964_4,9368.0,Winter,9,1,Student_School,No diploma yet,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31975,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,788821_1,41674.0,Winter,68,2,Retired,University,1,0.0
31976,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,791203_1,2846.0,Winter,83,1,Retired,University,1,0.0
31977,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,791203_2,3985.0,Winter,78,2,Retired,Secondary,1,0.0
31978,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,792023_1,3975.0,Winter,78,2,Retired,Secondary,1,0.0


In [53]:
df_UF=pd.concat([df_UF.drop(columns='Trip_Distance'),df_UF['Trip_Distance']],axis=1)
df_UF

,Res_geocode,DistSubcenter_res,DistCenter_res,UrbPopDensity_res,UrbBuildDensity_res,IntersecDensity_res,street_length_res,LU_UrbFab_res,bike_lane_share_res,LU_Comm_res,HH_PNR,Season,Age,Sex,Occupation,Education,CarOwnershipHH,Commute_Trip,Trip_Distance
0,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102956_1,Spring,29,2,Employed_PartTime,University,0,1.0,1458.0
1,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102964_1,Winter,46,1,Employed_PartTime,University,0,1.0,12518.0
2,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102964_2,Winter,46,2,Employed_PartTime,University,0,1.0,2948.0
3,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102964_3,Winter,12,1,Student_School,Elementary,0,0.0,9368.0
4,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102964_4,Winter,9,1,Student_School,No diploma yet,0,0.0,9368.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31975,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,788821_1,Winter,68,2,Retired,University,1,0.0,41674.0
31976,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,791203_1,Winter,83,1,Retired,University,1,0.0,2846.0
31977,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,791203_2,Winter,78,2,Retired,Secondary,1,0.0,3985.0
31978,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,792023_1,Winter,78,2,Retired,Secondary,1,0.0,3975.0


In [54]:
Occ_dict={'Employed_FullTime':'Employed','Employed_PartTime':'Employed','Employed':'Employed','Trainee':'Employed',
          'Student_School':'Student_School','Student_3rdLevel':'Student_3rdLevel','Pre-School':'Pre-School','Retired':'Retired',
          'Unemployed':'Unemployed/Other','Other':'Unemployed/Other','Home_Partner':'Unemployed/Other'}
Edu_dict={'University':'University','Secondary':'Secondary','Secondary+BAC':'Secondary','Secondary+Matura':'Secondary',
          'Apprenticeship':'Apprenticeship',
          'Elementary':'Primary/None','Pre-School':'Primary/None','No diploma yet':'Primary/None','Unknown':'Primary/None','Other':'Primary/None'}

df_UF['Occupation']=df_UF['Occupation'].map(Occ_dict)
df_UF['Education']=df_UF['Education'].map(Edu_dict)
df_UF

,Res_geocode,DistSubcenter_res,DistCenter_res,UrbPopDensity_res,UrbBuildDensity_res,IntersecDensity_res,street_length_res,LU_UrbFab_res,bike_lane_share_res,LU_Comm_res,HH_PNR,Season,Age,Sex,Occupation,Education,CarOwnershipHH,Commute_Trip,Trip_Distance
0,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102956_1,Spring,29,2,Employed,University,0,1.0,1458.0
1,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102964_1,Winter,46,1,Employed,University,0,1.0,12518.0
2,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102964_2,Winter,46,2,Employed,University,0,1.0,2948.0
3,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102964_3,Winter,12,1,Student_School,Primary/None,0,0.0,9368.0
4,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,102964_4,Winter,9,1,Student_School,Primary/None,0,0.0,9368.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31975,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,788821_1,Winter,68,2,Retired,University,1,0.0,41674.0
31976,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,791203_1,Winter,83,1,Retired,University,1,0.0,2846.0
31977,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,791203_2,Winter,78,2,Retired,Secondary,1,0.0,3985.0
31978,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,792023_1,Winter,78,2,Retired,Secondary,1,0.0,3975.0


In [56]:
df_UF.drop(columns=['HH_PNR'],inplace=True)
df_UF

,Res_geocode,DistSubcenter_res,DistCenter_res,UrbPopDensity_res,UrbBuildDensity_res,IntersecDensity_res,street_length_res,LU_UrbFab_res,bike_lane_share_res,LU_Comm_res,Season,Age,Sex,Occupation,Education,CarOwnershipHH,Commute_Trip,Trip_Distance
0,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,Spring,29,2,Employed,University,0,1.0,1458.0
1,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,Winter,46,1,Employed,University,0,1.0,12518.0
2,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,Winter,46,2,Employed,University,0,1.0,2948.0
3,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,Winter,12,1,Student_School,Primary/None,0,0.0,9368.0
4,10115,2.761334,1.972959,8794.155101,9.010181e+06,39.380867,127.710059,0.395459,0.0965,0.366144,Winter,9,1,Student_School,Primary/None,0,0.0,9368.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31975,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,Winter,68,2,Retired,University,1,0.0,41674.0
31976,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,Winter,83,1,Retired,University,1,0.0,2846.0
31977,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,Winter,78,2,Retired,Secondary,1,0.0,3985.0
31978,14199,1.863347,7.207826,5664.384999,2.971322e+06,46.726521,118.929373,0.555038,0.0712,0.098816,Winter,78,2,Retired,Secondary,1,0.0,3975.0


In [57]:
df_agg=df_UF.copy()
N=len(df_UF)

df_agg.sort_values(by='Res_geocode',inplace=True)
df_agg.dropna(subset=['Trip_Distance'],inplace=True)

target='Trip_Distance'

N_non_feature=1 # how many non-features are at the start of the df
cols=df_agg.columns
newcols=(df_agg.columns[:N_non_feature].tolist()) + ('FeatureD' +'_'+ cols[N_non_feature:-1]).tolist() + (df_agg.columns[-1:].tolist())
# change column names
df_agg.set_axis(newcols,axis=1,inplace=True)
df_agg = df_agg.reset_index(drop=True)

In [60]:
df0=df_agg.copy()

# convert  all categorical variables to dummies
df_Cat=df_agg.select_dtypes('object')[[col for col in df_agg.select_dtypes('object').columns if "FeatureD" in col]]
for col in df_Cat:
    dum=pd.get_dummies(df_agg[[col]])
    df_agg = pd.concat([df_agg, dum], axis = 1)
    # remove the original categorical columns
df_agg.drop(df_Cat.columns.tolist(),axis=1,inplace=True)


X=df_agg.drop(columns=['Res_geocode',target])
y=df_agg['Trip_Distance']

In [64]:
X.columns

Index(['FeatureD_DistSubcenter_res', 'FeatureD_DistCenter_res',
       'FeatureD_UrbPopDensity_res', 'FeatureD_UrbBuildDensity_res',
       'FeatureD_IntersecDensity_res', 'FeatureD_street_length_res',
       'FeatureD_LU_UrbFab_res', 'FeatureD_bike_lane_share_res',
       'FeatureD_LU_Comm_res', 'FeatureD_Age', 'FeatureD_Sex',
       'FeatureD_CarOwnershipHH', 'FeatureD_Commute_Trip',
       'FeatureD_Season_Autumn', 'FeatureD_Season_Spring',
       'FeatureD_Season_Summer', 'FeatureD_Season_Winter',
       'FeatureD_Occupation_Employed', 'FeatureD_Occupation_Pre-School',
       'FeatureD_Occupation_Retired', 'FeatureD_Occupation_Student_3rdLevel',
       'FeatureD_Occupation_Student_School',
       'FeatureD_Occupation_Unemployed/Other',
       'FeatureD_Education_Primary/None', 'FeatureD_Education_Secondary',
       'FeatureD_Education_University'],
      dtype='object')